In [0]:
#
# To activate this environment, use:
# > source activate py27
#
# To deactivate an active environment, use:
# > source deactivate
#
#https://github.com/minimaxir/facebook-page-post-scraper/blob/master/examples/how_to_build_facebook_scraper.ipynb

In [0]:
import urllib2
import json
import datetime
import csv
import time

In [0]:
app_id = "1427292924045575"
app_secret = "716ed65cf3992a3b1a467392ff46d405" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret

In [0]:
page_id = 'CincyPlanning'

In [0]:
def testFacebookPageData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    req = urllib2.Request(url)
    response = urllib2.urlopen(req)
    data = json.loads(response.read())
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageData(page_id, access_token)

{
    "id": "350608974951406", 
    "name": "Cincinnati City Planning"
}


In [0]:
def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)
            
            print "Error for URL %s: %s" % (url, datetime.datetime.now())

    return response.read()

In [0]:
def testFacebookPageFeedData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id + "/feed" # changed
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageFeedData(page_id, access_token)

{
    "data": [
        {
            "created_time": "2017-09-27T18:45:43+0000", 
            "id": "350608974951406_1653470227998601", 
            "message": "Join the City tomorrow evening at 6:00 p.m. at the Cincinnati Zoo for the kick-off meeting of the update for the Green Cincinnati Plan!"
        }, 
        {
            "created_time": "2017-09-26T19:25:36+0000", 
            "id": "350608974951406_1652658848079739", 
            "message": "Mark your calendars for a Kick-Off Meeting of the Oakley Master Plan on November 14th!  Also, please take the survey at https://www.surveymonkey.com/r/oaksurvey1"
        }, 
        {
            "created_time": "2017-09-25T19:05:14+0000", 
            "id": "350608974951406_1651783734833917", 
            "message": "Come join us tomorrow evening!", 
            "story": "Cincinnati City Planning shared Hyde Park Neighborhood Council's post."
        }, 
        {
            "created_time": "2017-09-20T19:56:28+0000", 
            "id

In [0]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data
    

test_status = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]
print json.dumps(test_status, indent=4, sort_keys=True)

{
    "comments": {
        "data": [], 
        "summary": {
            "can_comment": false, 
            "order": "ranked", 
            "total_count": 0
        }
    }, 
    "created_time": "2017-09-27T18:45:43+0000", 
    "id": "350608974951406_1653470227998601", 
    "likes": {
        "data": [
            {
                "id": "10212244927812239", 
                "name": "Katherine Keough Jurs"
            }
        ], 
        "paging": {
            "cursors": {
                "after": "MTAyMTIyNDQ5Mjc4MTIyMzkZD", 
                "before": "MTAyMTIyNDQ5Mjc4MTIyMzkZD"
            }, 
            "next": "https://graph.facebook.com/v2.10/350608974951406_1653470227998601/likes?access_token=1427292924045575%7C716ed65cf3992a3b1a467392ff46d405&summary=true&limit=1&after=MTAyMTIyNDQ5Mjc4MTIyMzkZD"
        }, 
        "summary": {
            "can_like": false, 
            "has_liked": false, 
            "total_count": 2
        }
    }, 
    "link": "http://www.cincinnati-o

In [0]:
def processFacebookPageFeedStatus(status):
    
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

processed_test_status = processFacebookPageFeedStatus(test_status)
print processed_test_status

(u'350608974951406_1653470227998601', 'Join the City tomorrow evening at 6:00 p.m. at the Cincinnati Zoo for the kick-off meeting of the update for the Green Cincinnati Plan!', 'Green Cincinnati Plan', u'link', u'http://www.cincinnati-oh.gov/oes/citywide-efforts/climate-protection-green-cincinnati-plan/', '2017-09-27 13:45:43', 2, 0, 2)


In [0]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_statuses.csv' % page_id, 'wb') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type", "status_link",
           "status_published", "num_likes", "num_comments", "num_shares"])
        
        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        
        print "Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime)
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                w.writerow(processFacebookPageFeedStatus(status))
                
                # output progress occasionally to make sure code is not stalling
                num_processed += 1
                if num_processed % 1000 == 0:
                    print "%s Statuses Processed: %s" % (num_processed, datetime.datetime.now())
                    
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
            else:
                has_next_page = False
                
        
        print "\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime)


scrapeFacebookPageFeedStatus(page_id, access_token)

Scraping CincyPlanning Facebook Page: 2017-10-02 09:42:53.438019



KeyError: 'next'